# Building the Image EXIF Extraction Skill
This notebook demonstrates how to use the Pida Simple REST API Service

In [286]:
#common setup
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#common imports
from cortex import Cortex
import os
import json
import re
import time
import datetime
%reload_ext cortex

## Building a Cortex Action

In [289]:
%%cortex_action --name 'demo/invoke_image_exif_api_rp' --function invoke_api
from cortex import Message, Cortex
import requests

def invoke_api(params):
    msg = Message(params)
    url = msg.payload.get('url')
    #sample payload
    #payload = {'key1': 'value1', 'key2': 'value2'}
    
    #get with no params on the input url
    r = requests.get(url)
    
    # GET with params in URL
    #r = requests.get(url, params=payload)

    return Message.with_payload({'response': r.text}).to_params()

Building Cortex Action (function): demo/invoke_image_exif_api_rp
Building Docker image private-registry.cortex.insights.ai/cortex-accelerators/demo_invoke_image_exif_api_rp:0z1ndy2...
Step 1/11 : FROM continuumio/miniconda3:4.5.4
Step 2/11 : WORKDIR /function
Step 3/11 : RUN apt-get update && apt-get install -y linux-headers-amd64 build-essential
Step 4/11 : RUN conda config --add channels conda-forge
Step 5/11 : COPY conda_requirements.txt .
Step 6/11 : RUN conda install --yes --file conda_requirements.txt
Step 7/11 : RUN pip install "dill==0.2.8.2" "fdk==0.0.31" "cortex-client==5.5.0"
Step 8/11 : COPY requirements.txt .
Step 9/11 : RUN pip install -r requirements.txt
Step 10/11 : COPY action.py .
Step 11/11 : ENTRYPOINT ["python", "action.py"]
Successfully built a5df653d03d4
Successfully tagged private-registry.cortex.insights.ai/cortex-accelerators/demo_invoke_image_exif_api_rp:0z1ndy2
Image private-registry.cortex.insights.ai/cortex-accelerators/demo_invoke_image_exif_api_rp:0z1ndy

Name,Version,Type,Kind,Image,Deployment Status,demo/invoke_image_exif_api_rp,1,function,python:3,private-registry.cortex.insights.ai/cortex-accelerators/demo_invoke_image_exif_api_rp:0z1ndy2,COMPLETED


### Testing Actions

In [360]:
# Instantiate Cortex Client
cortex = Cortex.client()
# Retrieve the Action that was deployed above
action = cortex.action('demo/invoke_image_exif_api_rp')

The Action deployment status should say **COMPLETED**.  This will indicate that our Action is ready to invoke.

In [364]:
action.get_deployment_status()

'COMPLETED'

Invoke the Action using a Message object.  Pass in the expected <Public_Image_URL> parameter in the Message payload to query a remote image URL.

In [367]:
from cortex import Message
# Sample Get Request
# https://www.pida.io/data/<Public_Image_URL>?format=json
rs = action.invoke(Message.with_payload({'url': "https://www.pida.io/data/https%3A%2F%2Fcogscalermp.s3.amazonaws.com%2FIMG_20190604_121215.jpg?format=json"}))
rs.payload

{'response': '{"FileType":"JPEG","FileTypeExtension":"jpg","MIMEType":"image/jpeg","ExifByteOrder":"Little-endian (Intel, II)","ExposureTime":"1/1661","FNumber":2,"ExposureProgram":"Program AE","ISO":49,"ExifVersion":"0220","DateTimeOriginal":"2019:06:04 12:12:15","CreateDate":"2019:06:04 12:12:15","ComponentsConfiguration":"Y, Cb, Cr, -","ShutterSpeedValue":"1/1663","ApertureValue":2,"BrightnessValue":8.73,"ExposureCompensation":0,"MaxApertureValue":2,"SubjectDistance":"0.783 m","MeteringMode":"Center-weighted average","Flash":"Off, Did not fire","FocalLength":"4.7 mm","SubSecTime":947158,"SubSecTimeOriginal":947158,"SubSecTimeDigitized":947158,"FlashpixVersion":"0100","ColorSpace":"sRGB","ExifImageWidth":3036,"ExifImageHeight":4048,"InteropIndex":"R98 - DCF basic file (sRGB)","InteropVersion":"0100","SensingMethod":"One-chip color area","SceneType":"Directly photographed","CustomRendered":"Custom","ExposureMode":"Auto","WhiteBalance":"Auto","DigitalZoomRatio":0,"FocalLengthIn35mmForm

## Building a Cortex Skill
Now that our Action is ready and tested, we can move on to building a Cortex Skill.  In this simple example, our Skill will just pipe an Input to our Action and route the Output back to the caller.

In [465]:
builder = cortex.builder()

Use the _skill_ method here to declare a new "Image EXIF Extract" Skill.  Each _builder_ method returns an instance of the builder so we can chain calls together.  

In [468]:
b = builder.skill('class/invoke_image_exif_api_rp').title('Invoke_Image_EXIF_API_RP').description('Image EXIF Extraction Skill by Rob Paisley')

Use the Input sub-builder to construct Skill Input; Declare how Input will route messages. Use _all_ routing to send all input messages to the same Action for processing and declare which Output to send Action outputs to. Pass in the Image EXIF Extraction Action from the previous section to wire the Skill to the Action (or use Action name). Call _build_ to create the input object, add it to the Skill builder, and return the Skill builder.

In [471]:
b = b.input('image_url').title('Image_URL').parameter(name='url', type='string').all_routing(action, 'exif_output').build()

Output called **exif_output**.  Create that Output using the Output sub-builder.

In [474]:
b = b.output('exif_output').title('EXIF_Output').parameter(name='exif', type='string').build()

## Preview the CAMEL Document
Preview the CAMEL using the _to_camel_ method.

In [477]:
b.to_camel()

{'camel': '1.0.0',
 'name': 'class/invoke_image_exif_api_rp',
 'title': 'Invoke_Image_EXIF_API_RP',
 'inputs': [{'name': 'image_url',
   'title': 'Image_URL',
   'parameters': [{'name': 'url', 'type': 'string', 'required': True}],
   'routing': {'all': {'action': 'demo/invoke_image_exif_api_rp',
     'output': 'exif_output'}}}],
 'outputs': [{'name': 'exif_output',
   'title': 'EXIF_Output',
   'parameters': [{'name': 'exif', 'type': 'string', 'required': True}]}],
 'description': 'Image EXIF Extraction Skill by Rob Paisley'}

## Final Build and Publish
Build and Publish Skill.

In [480]:
skill = b.build()
print('%s (%s) v%d' % (skill.title, skill.name, skill.version))

Invoke_Image_EXIF_API_RP (class/invoke_image_exif_api_rp) v1


In [501]:
rs = skill.invoke(input_name='image_exif_extract', message=Message.with_payload({'url': "https://www.pida.io/data/https%3A%2F%2Fcogscalermp.s3.amazonaws.com%2FIMG_20190604_121215.jpg?format=json"}))
rs.payload

Exception: No inputs defined